In [7]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display


# === ÉTAPE 1 : Charger et préparer le jeu de données brut =================================

# 1. Charger le fichier Excel contenant les actes
df = pd.read_excel("../data/2023_01_01_ExportRechLibre.xlsx")

# 2. Filtrer les lignes contenant des actes
df_actes = df.loc[df['Actes'].notna(), ['Identifiant passage unique', 'Actes']]

# 3. Extraire les actes un par un
event_log = []
pattern = r"(.*?)(\(\d{2}/\d{2}/\d{4} \d{2}:\d{2}\))"  # (activité) + (timestamp)

for _, row in df_actes.iterrows():
    case_id = row['Identifiant passage unique']
    texte = row['Actes']
    matches = re.findall(pattern, texte)

    for activity_raw, time_str in matches:
        activity = activity_raw.strip().lower()
        activity = re.sub(r"[^a-zA-Z0-9éèàçùêîôâ\- ]", "", activity)
        start_time = pd.to_datetime(time_str.strip("()"), format="%d/%m/%Y %H:%M")
        event_log.append({
            "case_id": case_id,
            "activity": activity,
            "start_timestamp": start_time
        })

event_log_df = pd.DataFrame(event_log)

# === ÉTAPE 2 : Préparer les catégories de référence =======================================

# 4. Charger les catégories avec exemples depuis le CSV
ref_df = pd.read_csv("../data/categories.csv", sep=";", encoding="utf-8-sig")
ref_df.dropna(inplace=True)

# 5. Nettoyer les noms de catégories
ref_df['category'] = ref_df['category'].astype(str).str.strip().str.lower()
ref_df['example'] = ref_df['example'].astype(str).str.strip().str.lower()

# === ÉTAPE 3 : Encodage des exemples et moyennes sémantiques ==============================

# 6. Charger le modèle d'embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# 7. Générer les vecteurs pour chaque exemple
ref_df['embedding'] = ref_df['example'].apply(lambda x: model.encode(x))

# 8. Calculer un vecteur moyen pour chaque catégorie
category_embeddings = (
    ref_df.groupby('category')['embedding']
    .apply(lambda x: np.mean(x.tolist(), axis=0))
    .to_dict()
)

# === ÉTAPE 4 : Classification des activités ===============================================

# 9. Encoder les activités à classer
event_log_df['activity_vector'] = event_log_df['activity'].apply(lambda x: model.encode(str(x)))

# 10. Fonction de classification
def classify_activity(vector):
    best_cat = None
    best_score = -1
    for category, ref_vector in category_embeddings.items():
        score = cosine_similarity([vector], [ref_vector])[0][0]
        if score > best_score:
            best_score = score
            best_cat = category
    return best_cat, best_score

# 11. Appliquer la classification
results = event_log_df['activity_vector'].apply(classify_activity)
event_log_df['predicted_category'] = results.apply(lambda x: x[0])
event_log_df['similarity_score'] = results.apply(lambda x: x[1])

# 12. Nettoyage 
event_log_df.drop(columns=['activity_vector'], inplace=True)

# === ÉTAPE 5 : Vérification et export ======================================================

# 13. Afficher les premiers résultats
event_log_df[['activity', 'predicted_category', 'similarity_score']].head(10)

display(event_log_df[['activity', 'predicted_category', 'similarity_score']].head(10))

# 14. Export des activités mal classées pour enrichissement manuel
ambigus = event_log_df[event_log_df['similarity_score'] < 0.75]
ambigus.to_csv("../data/actes_a_verifier.csv", index=False, encoding='utf-8-sig')

# 15. Sauvegarder le journal complet enrichi
event_log_df.to_csv("../data/event_log_classified.csv", index=False, encoding='utf-8-sig')


,activity,predicted_category,similarity_score
0,prise de la température,fecal,0.345811
1,mesure ponctuelle spo2,blood sampling,0.324586
2,crp capillaire resultat45,blood sampling,0.513540
3,pose de patch emla,drug dispensation,0.276443
4,désinfection plaie,cleaning,0.506807
5,aide à la suture,suture,0.494491
6,pose xylocaine,drug dispensation,0.371961
7,prise en charge en contexte traumatique yyyy011,fecal,0.387283
8,pansement moyen,tdr,0.367955
9,prise de la température,fecal,0.345811
